## Simulation as Optimization: particle simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from celluloid import Camera
from IPython.display import HTML
from base64 import b64encode

In [ ]:
def make_video(xs, path, interval=60, **kwargs): # xs: [time, N, 2]
    fig = plt.gcf() ; fig.set_dpi(100) ; fig.set_size_inches(3, 3)
    camera = Camera(fig)
    for i in range(xs.shape[0]):
        plt.plot(xs[i][...,0], xs[i][...,1], 'k.', markersize=20)
        plt.axis('equal') ; plt.xlim(0,1) ; plt.ylim(0,1)
#         plt.xticks([], []); plt.yticks([], [])
        camera.snap()
    anim = camera.animate(blit=True, interval=interval, **kwargs)
    anim.save(path) ; plt.close()

## Get a baseline simulation working

In [ ]:
N = 2
dt = 1
np.random.seed(6)
x0 = np.asarray([[0.4, 0.5], [0.6, 0.5]])
v0 = np.random.randn(N,2)*0
x1 = x0 + dt*v0

In [ ]:
# N = 6
# dt = 1
# np.random.seed(6)
# # x0 = np.asarray([[0.4, 0.5], [0.6, 0.5]])
# x0 = np.random.rand(N,2)*.5 + 0.25
# # v0 = -np.ones((N,2))*.025 ; v0[:,1] *= 0
# v0 = np.random.randn(N,2)*0
# x1 = x0 + dt*v0

In [ ]:
def potential_energy(xs, eps=1e-6, overlap_radius=0.05, scale_coeff=1e-4): # 1.5e-5
    if len(xs.shape) > 2:
        return sum([potential_energy(_xs, overlap_radius, scale_coeff) for _xs in xs]) # broadcast
    else:
        dist_matrix = ((xs[:,0:1] - xs[:,0:1].T).pow(2) + (xs[:,1:2] - xs[:,1:2].T).pow(2) + eps).sqrt()
        dists = dist_matrix[torch.triu_indices(N,N,1).split(1)]
        potentials =  (dists > overlap_radius) * 1/(dists + eps)  # 1/r^2
        potentials += (dists < overlap_radius) * (10*(overlap_radius - dists) + 1/overlap_radius)
        return - potentials.sum() * scale_coeff
    
def forces(xs, **kwargs):
    xs.requires_grad = True
    return torch.autograd.grad(potential_energy(xs), xs)[0]
    
print(potential_energy(torch.tensor(x0)))
forces(torch.tensor(x0))

In [ ]:
def particle_numerical(x0, x1, dt, steps=25, box_width=1):
    xs = [x0, x1]
    ts = [0, dt]
    v = (x1 - x0) / dt
    x = xs[-1]
    for i in range(steps-2):
        a = forces(torch.tensor(x)).numpy() # get forces/accelerations
        v = v + a*dt
        x = x + v*dt
        x = x % box_width
        xs.append(x)
        ts.append(ts[-1]+dt)
    return np.asarray(ts), np.stack(xs)

t_num, x_num = particle_numerical(x0, x1, dt)

In [ ]:
xs = x_num
path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Recover the same dynamics by minimizing the action

In [ ]:
def lagrangian(q, m=1, g=1):
    (x, xdot) = q
    T = .5*m*xdot**2
    V = potential_energy(x)
#     print(T.sum().item(), V.item())
    return T + V
  
def action(x, dt=1):
    dx = x[1:] - x[:-1]
    wraps = (dx.abs() > 0.9)
    dx = -dx.sign()*(dx.abs()-1)*wraps + dx*(~wraps)
    xdot = (dx) / dt
    xdot = torch.cat([xdot, xdot[-1:]], axis=0)
    return lagrangian(q=(x, xdot)).sum()

def get_path_between(x, steps=1000, step_size=1e-2, dt=1, box_width=1):
    t = np.linspace(0, len(x)-1, len(x)) * dt
    xs = [x.clone().data]
    for i in range(steps):
        grad = torch.autograd.grad(action(x, dt), x)
        grad_x = grad[0]
        grad_x[[0,-1]] *= 0
        x.data -= grad_x * step_size
        
        x = x % box_width # x is subject to modulo arithmetic

        if i % (steps//15) == 0:
            xs.append(x.clone().data)
            print('step={:04d}, S={:.3e}'.format(i, action(x).item()))
    return t, x, xs

N = 2
dt = 1
np.random.seed(6)
x0 = np.asarray([[0.4, 0.5],[0.6, 0.5]])
v0 = np.random.randn(N,2)*0
x1 = x0 + dt*v0
t_num, x_num = particle_numerical(x0, x1, dt)
x_sim = x_num
x_num = xs_sim #xs_nopot

x_noise = .005*np.random.randn(*x_num.shape).clip(-1,1)
x_noise[:1] = x_noise[-1:] = 0
x_pert = (x_num + x_noise).reshape(-1, N*2)
x0 = torch.tensor(x_pert, requires_grad=True) # [time, N*2]
t_min, x_min, xs_min = get_path_between(x0)

In [ ]:
xs_before = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_after = xs_min[-1].detach().numpy().reshape(-1,N,2)

plt.figure(dpi=100)
plt.title('Ball 2 horiz. velocity vs. time (R=0.025)')
plt.plot((xs_before[1:] - xs_before[:-1]).reshape(-1,4)[:-1,0], '.-', label='Initial path')
plt.plot((xs_after[1:] - xs_after[:-1]).reshape(-1,4)[:-1,0], '.-', label='Minimum action')
plt.plot((xs_nopot[1:] - xs_nopot[:-1]).reshape(-1,4)[:-1,0], label='Minimum action, no potential')
plt.plot((xs_sim[1:] - xs_sim[:-1]).reshape(-1,4)[:-1,0], 'k-', label='Simulator')
plt.legend()
plt.show()

In [ ]:
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)

path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url) # min_action_no_forces

In [ ]:
xs_sim = xs_before

In [ ]:
xs_nopot = xs_after

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
plt.title('Ball 2 horiz. velocity vs. time [before minimizing action]')
plt.plot((xs[1:] - xs[:-1]).reshape(-1,4)[:-1,2])

In [ ]:
# no potential term

In [ ]:
xs = xs_min[0].detach().numpy().reshape(-1,N,2)
xs = x_num

path = 'sim.mp4' ; make_video(xs, path, interval=60)
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=300 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# def _potential_energy(xs, do_overlap=False, eps=1e-6, overlap_radius=0.15):
#     dist_matrix = ((xs[:,0:1] - xs[:,0:1].T)**2 + (xs[:,1:2] - xs[:,1:2].T)**2)**.5  # distance matrix
# #     dist_matrix[torch.triu_indices(N,N,0).split(1)] = 100
#     print(dist_matrix)
#     return (dist_matrix + eps).sum()
# #     return (1/(dist_matrix + eps)).sum()
# #     dist_matrix[torch.triu_indices(N,N,0).split(1)] = torch.inf  # we only care about lower triangle
# #     is_overlapping = (dist_matrix < overlap_radius)
# #     potentials = ~is_overlapping * 1/(dist_matrix + eps)  # 1/r^2 potential if not overlapping
# #     if do_overlap:
# #         dist_matrix[torch.triu_indices(N,N,0).split(1)] = 0  # now these need to be zero
# #         potentials += is_overlapping * (5*(overlap_radius - dist_matrix) + 1/overlap_radius) # c*(R-r)
# #     return potentials.sum()

# def potential_energy(xs, **kwargs):
#     if type(xs) == np.ndarray:
#         xs = torch.tensor(xs)
#     if len(xs.shape) > 2:
#         return sum([potential_energy(_xs, **kwargs) for _xs in xs]) # broadcast across additional axes
#     else:
#         return _potential_energy(xs, **kwargs)
    
# potential_energy(x_num[0], do_overlap=True)